#### Script para recortar e poligonizar a base de Declividade por Munícipios

Projeto: Sistema de Apoio à Caracterização de Imóveis Rurais  
Embrapa/2023

In [1]:
# Módulos necessários
import os
import glob
import rasterio
from rasterio.mask import mask
import shapely
import geopandas as gpd
import numpy as np
from osgeo import ogr, osr, gdal
import time
import shutil
import pyproj

In [2]:
# Definir diretório principal
dirpath = input('Diretório principal: ')

In [ ]:
# Definir as pastas com as declividades originais
originais_path = input('Pasta com originais: ')

In [3]:
# Definir pasta com os arquivos do CAR
car_path = input('Diretório do CAR: ')

In [ ]:
# Definir caminho do shapefile que será usado como máscara para recorte (Municípios do BR - Buffer de 150 metros)
limites_buf = input('Caminho Municípios com Buffer: ')

In [ ]:
# Definir caminho do shapefile que será usado como máscara para recorte (Municípios do BR - sem Buffer)
limites = input('Caminho Municípios: ')

In [4]:
# Criar pasta para armazenar os vetores de Declividade por Município
out_path = os.path.join(dirpath, 'Decliv_UF_Municipios')
os.makedirs(out_path, exist_ok=True)
# Criar pasta para armazenar arquivos temporários
temp_path = os.path.join(dirpath, 'Temporarios')
os.makedirs(temp_path, exist_ok=True)
# Lendo shapefile que será usado como máscara para recorte (com buffer)
lim_buf = gpd.read_file(rf"{limites_buf}")
# Reprojetando o shapefile para WGS84
lim_buf = lim_buf.to_crs(epsg=4326)
# Lendo shapefile que será usado como máscara para recorte (sem buffer)
lim = gpd.read_file(rf"{limites}")
# Reprojetando o shapefile para WGS84
lim = lim.to_crs(epsg=4326)
# Lista dos estados
estados = list(np.unique(lim['SIGLA_UF']))
# Recortando os rasters de declividade pelo buffer dos municípios
for estado in estados:
    rasters = glob.glob(originais_path + fr'**/{estado}.tif')
    # Selecionar GeoDataFrame por estado
    select = lim_buf[lim_buf['SIGLA_UF'] == f'{estado}']
    # Códigos dos municípios
    names = [x for x in select['CD_MUN']]
    for raster in rasters:
        # Lendo o raster
        ras_data = rasterio.open(raster)
        # Definindo o caminho de saída do raster recortado
        nome_arquivo = os.path.basename(raster)
        nome_arquivo = nome_arquivo.replace('.tif', '')
        output = os.path.join(temp_path, nome_arquivo)
        # Recortando o raster por município
        for i in range(len(select)):
            geom = []
            coord = shapely.geometry.mapping(select)["features"][i]["geometry"]
            geom.append(coord)
            with rasterio.open(raster) as src:
                out_image, out_transform = rasterio.mask.mask(src,geom,crop=True)
                out_meta = src.meta
            out_meta.update({'driver':'GTiff',
                            'height':out_image.shape[1],
                            'width':out_image.shape[2],
                            'transform':out_transform})
            # Salvando os rasters recortados
            with rasterio.open(f'{output}_{names[i]}_temp1.tif','w',**out_meta)as dest: dest.write(out_image)
# Buscar imagens recortadas
imagens = glob.glob(temp_path + '**/*temp1.tif')
# Convertendo Raster para Shapefile
for imagem in imagens:
    driver = gdal.GetDriverByName('GTiff')
    input = gdal.Open(imagem)
    band = input.GetRasterBand(1)
    band1 = band.ReadAsArray()
    proj = input.GetProjection()
    shp_proj = osr.SpatialReference()
    shp_proj.ImportFromWkt(proj)
    nome_arq = os.path.basename(imagem).replace('temp1.tif', 'temp2.shp')
    output = temp_path + f"\{nome_arq}"
    call_drive = ogr.GetDriverByName('ESRI Shapefile')
    create_shp = call_drive.CreateDataSource(output)
    shp_layer = create_shp.CreateLayer('Declividade', srs = shp_proj)
    new_field = ogr.FieldDefn(str('CD_DECLIV'), ogr.OFTInteger)
    shp_layer.CreateField(new_field)
    gdal.Polygonize(band, None, shp_layer, 0, [], callback = None)
    create_shp.Destroy()
    raster = None
# Dissolve
imagens = glob.glob(temp_path + '**/*temp2.shp')
for imagem in imagens:
    geodf = gpd.read_file(imagem)
    dissolve = geodf.dissolve(by='CD_DECLIV')
    nome_arq = os.path.basename(imagem).replace('temp2.shp', 'temp3.shp')
    output = temp_path + f"\{nome_arq}"
    dissolve.to_file(driver = 'ESRI Shapefile', filename = output)
# Cortar por município
# Agrupar por município
agrupado = lim.groupby('CD_MUN')
# Shapefiles que serão recortados
shapes = glob.glob(temp_path + '**/*temp3.shp')
for shape in shapes:
    for key,values in agrupado:
        if key in shape:
            decliv = gpd.read_file(shape)
            decliv = decliv.to_crs(epsg=4326)
            nome_arq = os.path.basename(shape).replace("_temp3.shp", ".shp")
            output = out_path + f"\Decliv_{nome_arq}"
            mun = lim[lim["CD_MUN"] == f"{key}"]
            geodf_clip = gpd.clip(decliv, mun)
            geodf_clip.to_file(driver = 'ESRI Shapefile', filename = (rf'{output}'))

In [8]:
# Renomeando AREA_IMOVEL para seus respectivos municípios
municipios = []
# Percorre a pasta 'CAR' e suas subpastas
for root, dirs, files in os.walk(car_path):
    for dir in dirs:
        # Verifica se o nome da subpasta é um código de município
        if dir.startswith('SHAPE_'):
            municipio_codigo = dir.split('_')[1]
            municipios.append(municipio_codigo)
            # Constrói o caminho completo para a pasta 'AREA_IMOVEL'
            area_imovel_dir = os.path.join(root, dir, 'AREA_IMOVEL')
            
            # Verifica se a pasta 'AREA_IMOVEL' existe
            if os.path.exists(area_imovel_dir):
                # Renomeia os arquivos dentro de 'AREA_IMOVEL'
                for filename in os.listdir(area_imovel_dir):
                    if filename.startswith('AREA_IMOVEL'):
                        novo_nome = f'{municipio_codigo}_{filename}'
                        arquivo_antigo = os.path.join(area_imovel_dir, filename)
                        novo_arquivo = os.path.join(area_imovel_dir, novo_nome)
                        os.rename(arquivo_antigo, novo_arquivo)
                        print(f'Renomeado: {arquivo_antigo} -> {novo_arquivo}')
# Criar pasta para armazenar os arquivos finais de Declividade por município integrados com CAR
final_path = os.path.join(dirpath, 'Decliv_UF_Municipios_CAR')
os.makedirs(final_path, exist_ok=True)
# Integração com CAR
# Caminho dos vetores de Declividade
decl = glob.glob(out_path + fr'**/*.shp')
# Caminho dos vetores de limites dos imóveis
car = glob.glob(car_path + '**/*/*/*/*.shp')
caminho_decl = []
caminho_car = []
for arquivo in decl:
    caminho = os.path.join(out_path,arquivo)
    caminho_decl.append(caminho)
for arquivo in car:
    caminho = os.path.join(car_path,arquivo)
    caminho_car.append(caminho)
vetores = caminho_decl + caminho_car
for municipio in municipios:
    for vetor in vetores:
        if municipio in vetor and 'Decliv' in vetor:
            nome_arq = os.path.basename(vetor).replace('.shp', '_CAR.shp')
            uso = gpd.read_file(vetor)
        if municipio in vetor and 'AREA_IMOVEL' in vetor:
            area = gpd.read_file(vetor)
    base_car = gpd.overlay(uso, area, how='intersection', keep_geom_type=True)
    base_car.to_file(driver = 'ESRI Shapefile', filename = (rf'{final_path}\{nome_arq}'))

C:\Users\youx\AppData\Local\Temp\ipykernel_15732\2827011907.py:47: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4674

  base_car = gpd.overlay(uso, area, how='intersection', keep_geom_type=True)
C:\Users\youx\AppData\Local\Temp\ipykernel_15732\2827011907.py:47: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4674

  base_car = gpd.overlay(uso, area, how='intersection', keep_geom_type=True)
